In [20]:
import os
import pandas as pd

In [21]:
df = pd.read_csv("../data_given/healthcare-dataset-stroke-data.csv")

In [22]:
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [23]:
df.columns

Index(['id', 'gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status', 'stroke'],
      dtype='object')

In [24]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,5110.0,36517.829354,21161.721625,67.00,17741.250,36932.000,54682.00,72940.00
age,5110.0,43.226614,22.612647,0.08,25.000,45.000,61.00,82.00
hypertension,5110.0,0.097456,0.296607,0.00,0.000,0.000,0.00,1.00
heart_disease,5110.0,0.054012,0.226063,0.00,0.000,0.000,0.00,1.00
avg_glucose_level,5110.0,106.147677,45.283560,55.12,77.245,91.885,114.09,271.74
bmi,4909.0,28.893237,7.854067,10.30,23.500,28.100,33.10,97.60
stroke,5110.0,0.048728,0.215320,0.00,0.000,0.000,0.00,1.00


In [25]:
# df.describe().loc[['min','max']].to_json("schema_in.json")

In [26]:
# class NotInRange(Exception):
#     def __init__(self,message="value not in valid range"):
#         self.message = message
#         super().__init__(self.message)

In [27]:
# raise NotInRange

In [28]:
df.drop("id",axis=1)

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
5105,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [29]:
df.count()

id                   5110
gender               5110
age                  5110
hypertension         5110
heart_disease        5110
ever_married         5110
work_type            5110
Residence_type       5110
avg_glucose_level    5110
bmi                  4909
smoking_status       5110
stroke               5110
dtype: int64

In [30]:
df.isnull().sum()

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

In [31]:
(201/5110)*100

3.9334637964774952

In [91]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

In [33]:
df_with_bmi = df[df['bmi'].notnull()]

In [34]:
df_without_bmi = df[df['bmi'].isnull()]


In [37]:
features = ['age', 'avg_glucose_level','heart_disease','Residence_type','work_type', 'gender','stroke','smoking_status']

In [38]:
X = pd.get_dummies(df_with_bmi[features], drop_first=True)
y = df_with_bmi['bmi']

In [39]:
X.head()

,age,avg_glucose_level,heart_disease,stroke,Residence_type_Urban,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,gender_Male,gender_Other,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,228.69,1,1,True,False,True,False,False,True,False,True,False,False
2,80.0,105.92,1,1,False,False,True,False,False,True,False,False,True,False
3,49.0,171.23,0,1,True,False,True,False,False,False,False,False,False,True
4,79.0,174.12,0,1,False,False,False,True,False,False,False,False,True,False
5,81.0,186.21,0,1,True,False,True,False,False,True,False,True,False,False


In [41]:
df['gender'].unique()

array(['Male', 'Female', 'Other'], dtype=object)

In [42]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [89]:
bmi_predictor = XGBRegressor(n_estimators=100, learning_rate=0.8, max_depth=100, random_state=42)
bmi_predictor.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.8, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=100, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [90]:
y_pred = bmi_predictor.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
print(f"Validation MSE: {mse}")

Validation MSE: 78.37857614136132


In [99]:
y_pred = bmi_predictor.predict(X_train)
mse = mean_squared_error(y_train, y_pred)
print(f"Trining MSE: {mse}")

Trining MSE: 0.23532095922971913


In [47]:
from sklearn.linear_model import ElasticNet

In [75]:
elastic_net = ElasticNet(alpha=0.7, l1_ratio=0.43, random_state=42)  # l1_ratio balances Lasso and Ridge penalties
elastic_net.fit(X_train, y_train)

ElasticNet(alpha=0.7, l1_ratio=0.43, random_state=42)

In [76]:
y_pred = elastic_net.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
print(f"Validation MSE: {mse}")

Validation MSE: 59.85022084176349


In [93]:
param_grid = {
    'n_estimators': [50, 100, 150,200],
    'max_depth': [5, 10, 20,40,70,100,150],
    'learning_rate': [0.01, 0.1, 0.3, 0.5,0,6,0.8]
}

In [94]:
grid_search = GridSearchCV(estimator=bmi_predictor, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)

# Step 6: Fit GridSearchCV to the training data
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 196 candidates, totalling 588 fits


C:\Users\kevin\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_search.py:1051: UserWarning: One or more of the test scores are non-finite: [-4.97615781e+01 -4.59709464e+01 -4.47041363e+01 -4.43734044e+01
 -5.05160567e+01 -4.75785060e+01 -4.70278011e+01 -4.75598429e+01
 -5.26325624e+01 -5.20373513e+01 -5.34048273e+01 -5.49399154e+01
 -5.30191163e+01 -5.29916369e+01 -5.46692039e+01 -5.66204012e+01
 -5.30191163e+01 -5.29916369e+01 -5.46692039e+01 -5.66204012e+01
 -5.30191163e+01 -5.29916369e+01 -5.46692039e+01 -5.66204012e+01
 -5.30191163e+01 -5.29916369e+01 -5.46692039e+01 -5.66204012e+01
 -4.49517564e+01 -4.63609291e+01 -4.77139895e+01 -4.89855758e+01
 -5.12706114e+01 -5.51056017e+01 -5.69579743e+01 -5.78925052e+01
 -6.07696483e+01 -6.23470352e+01 -6.25059149e+01 -6.25259603e+01
 -6.17777379e+01 -6.30550458e+01 -6.31451620e+01 -6.31492054e+01
 -6.17777379e+01 -6.30546617e+01 -6.31467229e+01 -6.31507275e+01
 -6.17777379e+01 -6.30546617e+01 -6.31467229e+01 -6.31507

GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=0.8, ma...
                                    max_depth=100, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=42, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.3, 0.5, 0, 6, 0.8],
                         'max_depth': [5, 10, 20, 40, 70, 100, 150],
                         'n_estimators': [50, 100, 150, 200]},
             scoring='neg_mean_squared_error', verbose=1)

In [95]:
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 196 candidates, totalling 588 fits


C:\Users\kevin\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_search.py:1051: UserWarning: One or more of the test scores are non-finite: [-4.97615781e+01 -4.59709464e+01 -4.47041363e+01 -4.43734044e+01
 -5.05160567e+01 -4.75785060e+01 -4.70278011e+01 -4.75598429e+01
 -5.26325624e+01 -5.20373513e+01 -5.34048273e+01 -5.49399154e+01
 -5.30191163e+01 -5.29916369e+01 -5.46692039e+01 -5.66204012e+01
 -5.30191163e+01 -5.29916369e+01 -5.46692039e+01 -5.66204012e+01
 -5.30191163e+01 -5.29916369e+01 -5.46692039e+01 -5.66204012e+01
 -5.30191163e+01 -5.29916369e+01 -5.46692039e+01 -5.66204012e+01
 -4.49517564e+01 -4.63609291e+01 -4.77139895e+01 -4.89855758e+01
 -5.12706114e+01 -5.51056017e+01 -5.69579743e+01 -5.78925052e+01
 -6.07696483e+01 -6.23470352e+01 -6.25059149e+01 -6.25259603e+01
 -6.17777379e+01 -6.30550458e+01 -6.31451620e+01 -6.31492054e+01
 -6.17777379e+01 -6.30546617e+01 -6.31467229e+01 -6.31507275e+01
 -6.17777379e+01 -6.30546617e+01 -6.31467229e+01 -6.31507

GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=0.8, ma...
                                    max_depth=100, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=42, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.3, 0.5, 0, 6, 0.8],
                         'max_depth': [5, 10, 20, 40, 70, 100, 150],
                         'n_estimators': [50, 100, 150, 200]},
             scoring='neg_mean_squared_error', verbose=1)

In [96]:
print(f"Best Parameters: {grid_search.best_params_}")
best_model = grid_search.best_estimator_

Best Parameters: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 200}


In [97]:
y_pred = best_model.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
print(f"Validation MSE: {mse}")


Validation MSE: 52.60406858689175


In [100]:
y_pred = best_model.predict(X_train)
mse = mean_squared_error(y_train, y_pred)
print(f"Trining MSE: {mse}")

Trining MSE: 41.04244630379759


In [101]:

dummies = pd.get_dummies(df[features], drop_first=True)

# Align the columns of your missing data with the trained model's feature set
X_missing = dummies.loc[df['bmi'].isnull()]

# Now, you can make predictions
df.loc[df['bmi'].isnull(), 'bmi'] = best_model.predict(X_missing)


In [102]:
df['bmi'].isnull().sum()

0

In [103]:
df.isnull().sum()

id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [104]:
df[df['stroke']==1].count()

id                   249
gender               249
age                  249
hypertension         249
heart_disease        249
ever_married         249
work_type            249
Residence_type       249
avg_glucose_level    249
bmi                  249
smoking_status       249
stroke               249
dtype: int64

In [105]:
df[df['stroke']==0].count()

id                   4861
gender               4861
age                  4861
hypertension         4861
heart_disease        4861
ever_married         4861
work_type            4861
Residence_type       4861
avg_glucose_level    4861
bmi                  4861
smoking_status       4861
stroke               4861
dtype: int64

In [106]:
print('Inverse of Null Accuracy: ',249/(249+4861))
print('Null Accuracy: ',4861/(4861+249))

Inverse of Null Accuracy:  0.0487279843444227
Null Accuracy:  0.9512720156555773


In [121]:
df['smoking_status'].unique()

array(['formerly smoked', 'never smoked', 'smokes', 'Unknown'],
      dtype=object)

In [125]:
df['gender'] = df['gender'].replace({'Male':0,'Female':1,'Other':-1}).astype(np.uint8)
df['smoking_status'] = df['smoking_status'].replace({'Unknown':0,'never smoked':1,'formerly smoked':2,"smokes":3}).astype(np.uint8)
df['Residence_type'] = df['Residence_type'].replace({'Rural':0,'Urban':1}).astype(np.uint8)

df['work_type'] = df['work_type'].replace({'Private':0,'Self-employed':1,'Govt_job':2,'children':-1,'Never_worked':-2}).astype(np.uint8)

C:\Users\kevin\AppData\Local\Temp\ipykernel_14468\1396093129.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['smoking_status'] = df['smoking_status'].replace({'Unknown':0,'never smoked':1,'formerly smoked':2,"smokes":3}).astype(np.uint8)


In [126]:
df.drop("id",inplace=True,axis=1)

KeyError: "['id'] not found in axis"

In [127]:
X  = df.drop(["stroke","ever_married"],axis=1)
y = df['stroke']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.3, random_state=42)

In [128]:
X.head()

,gender,age,hypertension,heart_disease,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,0,67.0,0,1,0,1,228.69,36.600000,2
1,1,61.0,0,0,1,0,202.21,34.860165,1
2,0,80.0,0,1,0,0,105.92,32.500000,1
3,1,49.0,0,0,0,1,171.23,34.400000,3
4,1,79.0,1,0,1,0,174.12,24.000000,1


In [129]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_train_resh, y_train_resh = oversample.fit_resample(X_train, y_train.ravel())

C:\Users\kevin\AppData\Local\Temp\ipykernel_14468\3838932189.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  X_train_resh, y_train_resh = oversample.fit_resample(X_train, y_train.ravel())


In [138]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.svm import SVC


from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, precision_score, f1_score

In [140]:
rf = Pipeline(steps = [("scale",StandardScaler()),("RF",RandomForestClassifier(random_state=42))])
svm = Pipeline(steps = [("scale",StandardScaler()),("SVM",SVC(random_state=42))])
lr = Pipeline(steps = [("scale",StandardScaler()),("RF",LogisticRegression(random_state=42))])


In [142]:
rf_cv = cross_val_score(rf,X_train_resh,y_train_resh,cv=10,scoring='f1')
svm_cv = cross_val_score(svm,X_train_resh,y_train_resh,cv=10,scoring='f1')
logreg_cv = cross_val_score(lr,X_train_resh,y_train_resh,cv=10,scoring='f1')

In [143]:
print('Mean f1 scores:')
print('Random Forest mean :',rf_cv.mean())
print('SVM mean :',svm_cv.mean())
print('Logistic Regression mean :',logreg_cv.mean())

Mean f1 scores:
Random Forest mean : 0.9490554564776911
SVM mean : 0.8757953022637514
Logistic Regression mean : 0.8312943417841678


In [145]:

rfc = RandomForestClassifier(max_features=2,n_estimators=100,bootstrap=True)

rfc.fit(X_train_resh,y_train_resh)

rfc_tuned_pred = rfc.predict(X_test)

In [146]:
print(classification_report(y_test,rfc_tuned_pred))

print('Accuracy Score: ',accuracy_score(y_test,rfc_tuned_pred))
print('F1 Score: ',f1_score(y_test,rfc_tuned_pred))

              precision    recall  f1-score   support

           0       0.96      0.93      0.94      3404
           1       0.11      0.17      0.13       173

    accuracy                           0.89      3577
   macro avg       0.53      0.55      0.54      3577
weighted avg       0.92      0.89      0.90      3577

Accuracy Score:  0.8912496505451496
F1 Score:  0.133630289532294


In [151]:
import seaborn as sns  
import matplotlib.pyplot as plt
import matplotlib

In [153]:
lr = Pipeline(steps = [('scale',StandardScaler()),('LR',LogisticRegression(C=0.1,penalty='l2',random_state=42))])

lr.fit(X_train_resh,y_train_resh)

#logreg.fit(X_train_resh,y_train_resh)

logreg_tuned_pred   = lr.predict(X_test)
print(classification_report(y_test,logreg_tuned_pred))

print('Accuracy Score: ',accuracy_score(y_test,logreg_tuned_pred))
print('F1 Score: ',f1_score(y_test,logreg_tuned_pred))

              precision    recall  f1-score   support

           0       0.97      0.77      0.86      3404
           1       0.12      0.58      0.19       173

    accuracy                           0.76      3577
   macro avg       0.54      0.68      0.53      3577
weighted avg       0.93      0.76      0.83      3577

Accuracy Score:  0.7648867766284596
F1 Score:  0.19212295869356388


In [156]:
df.drop("ever_married",inplace=True,axis=1)


In [157]:
df.head()

,gender,age,hypertension,heart_disease,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,67.0,0,1,0,1,228.69,36.600000,2,1
1,1,61.0,0,0,1,0,202.21,34.860165,1,1
2,0,80.0,0,1,0,0,105.92,32.500000,1,1
3,1,49.0,0,0,0,1,171.23,34.400000,3,1
4,1,79.0,1,0,1,0,174.12,24.000000,1,1


In [158]:
df.describe().loc[['min','max']].to_json("schema_in.json")